<h1 style="color:DodgerBlue">Индивидальный проект</h1>

<h2 style="color:DodgerBlue">Название проекта:</h2>

----

### Вариант 16 


<h2 style="color:DodgerBlue">Описание проекта:</h2>

----
Описание задачи:
Создать базовый класс PaymentMethod в C#, который будет представлять
различные способы оплаты. На основе этого класса разработать 2-3 производных
класса, демонстрирующих принципы наследования и полиморфизма. В каждом из
классов должны быть реализованы новые атрибуты и методы, а также
переопределены некоторые методы базового класса для демонстрации
полиморфизма.

Требования к базовому классу PaymentMethod:

• Атрибуты: ID способа оплаты (PaymentMethodId), Название способа оплаты
(MethodName), Минимальная сумма (MinAmount).

• Методы:
o ProcessPayment(decimal amount): метод для обработки платежа
указанной суммы.
o CheckMinimumAmount(decimal amount): метод для проверки
минимальной суммы платежа.
o GetPaymentDetails(): метод для получения деталей способа оплаты.

Требования к производным классам:
1. ОнлайнОплата (OnlinePayment): Должен содержать дополнительные
атрибуты, такие как URL платежной системы (PaymentUrl).
Метод ProcessPayment() должен быть переопределен для включения URL
платежной системы в процесс оплаты.
2. БанковскийПеревод (BankTransfer): Должен содержать дополнительные
атрибуты, такие как Банковские данные (BankData).
Метод CheckMinimumAmount() должен быть переопределен для проверки
минимальной суммы платежа с учетом банковских комиссий.
3. Наличные (CashPayment) (если требуется третий класс): Должен содержать
дополнительные атрибуты, такие как Место выдачи наличных
(CashPickupPoint). Метод GetPaymentDetails() должен быть переопределен
для отображения места выдачи наличных.

#### Дополнительное задание
Добавьте к сущестующим классам (базовыму и производным 3-4 атрибута и метода) создайте явную реализации интерфейса и управление зависимостями 


<h2 style="color:DodgerBlue">Реализация:</h2>

----

In [ ]:
using System;
using System.Collections.Generic;

// Интерфейс для логирования
public interface ILogger
{
    void Log(string message);
}

// Реализация логирования в консоль
public class ConsoleLogger : ILogger
{
    public void Log(string message) => Console.WriteLine(message);
}

// Интерфейс для генерации ID транзакций
public interface ITransactionIdGenerator
{
    string GenerateTransactionId();
}

// Реализация генератора ID транзакций (Руководство)
public class GuidTransactionIdGenerator : ITransactionIdGenerator
{
    public string GenerateTransactionId() => Guid.NewGuid().ToString();
}

// Интерфейс для PaymentMethod
public interface IPaymentMethod
{
    int PaymentMethodId { get; }
    string MethodName { get; }
    decimal MinAmount { get; }
    DateTime PaymentDate { get; }
    string Description { get; set; }
    bool IsCompleted { get; set; }
    string Currency { get; }
    List<PaymentHistory> PaymentHistory { get; }
    void ProcessPayment(decimal amount);
    bool CheckMinimumAmount(decimal amount);
    string GetPaymentDetails();
    List<PaymentHistory> GetPaymentHistory();
}


// Базовый класс PaymentMethod
public class PaymentMethod : IPaymentMethod
{
    private int _paymentMethodId;
    private string _methodName;
    private decimal _minAmount;
    private DateTime _paymentDate;
    private string _description;
    private bool _isCompleted;
    private string _currency;
    private List<PaymentHistory> _paymentHistory;
    private string _processingFeeCurrency; // Добавлена: Валюта комиссии за обработку
    private decimal _processingFee; // Добавлена: Сумма комиссии за обработку
    protected readonly ILogger _logger;
    protected readonly ITransactionIdGenerator _transactionIdGenerator;


    public int PaymentMethodId { get => _paymentMethodId; set => _paymentMethodId = value; }
    public string MethodName { get => _methodName; set => _methodName = value; }
    public decimal MinAmount { get => _minAmount; set => _minAmount = value; }
    public DateTime PaymentDate { get => _paymentDate; set => _paymentDate = value; }
    public string Description { get => _description; set => _description = value; }
    public bool IsCompleted { get => _isCompleted; set => _isCompleted = value; }
    public string Currency { get => _currency; set => _currency = value; }
    public List<PaymentHistory> PaymentHistory { get => _paymentHistory; set => _paymentHistory = value; }
    public string ProcessingFeeCurrency { get => _processingFeeCurrency; set => _processingFeeCurrency = value; } // Добавлено
    public decimal ProcessingFee { get => _processingFee; set => _processingFee = value; } // Добавлено

    public PaymentMethod(int paymentMethodId, string methodName, decimal minAmount, string currency, ILogger logger, ITransactionIdGenerator transactionIdGenerator)
    {
        PaymentMethodId = paymentMethodId;
        MethodName = methodName;
        MinAmount = minAmount;
        Currency = currency;
        PaymentHistory = new List<PaymentHistory>();
        _logger = logger;
        _transactionIdGenerator = transactionIdGenerator;
    }

    public virtual void ProcessPayment(decimal amount)
    {
        if (CheckMinimumAmount(amount))
        {
            _logger.Log($"Обработка платежа методом {MethodName} на сумму {amount} {Currency}");
            PaymentDate = DateTime.Now;
            IsCompleted = true;
            AddPaymentHistory(amount);
        }
    }

    public virtual bool CheckMinimumAmount(decimal amount)
    {
        if (amount < MinAmount)
        {
            _logger.Log($"Сумма платежа меньше минимальной ({MinAmount} {Currency})");
            return false;
        }
        return true;
    }

    public virtual string GetPaymentDetails()
    {
        return $"Способ оплаты: {MethodName}, Минимальная сумма: {MinAmount} {Currency}";
    }

    public virtual List<PaymentHistory> GetPaymentHistory() => PaymentHistory;

    private void AddPaymentHistory(decimal amount)
    {
        PaymentHistory.Add(new PaymentHistory
        {
            PaymentId = PaymentHistory.Count + 1,
            PaymentMethod = MethodName,
            Amount = amount,
            PaymentDate = PaymentDate
        });
    }

    // Добавлен: Способ взимания платы за обработку
    public virtual decimal ApplyProcessingFee(decimal amount)
    {
        decimal fee = amount * (ProcessingFee / 100); // Предполагая, что коэффициент обработки равен проценту
        _logger.Log($"Взимается плата за обработку: {fee} {ProcessingFeeCurrency}");
        return fee;
    }

    // Добавлен: Способ получения статуса платежа
    public virtual string GetPaymentStatus() => IsCompleted ? "Завершенный" : "Ожидаемый";
}


// Производный класс ОнлайнОплата
public class OnlinePayment : PaymentMethod
{
    private string _paymentUrl;
    private string _transactionId;
    private int _attemptsCount;
    private DateTime _lastAttemptTime; // Добавлено: Время последней попытки

    public string PaymentUrl { get => _paymentUrl; set => _paymentUrl = value; }
    public string TransactionId { get => _transactionId; set => _transactionId = value; }
    public int AttemptsCount { get => _attemptsCount; set => _attemptsCount = value; }
    public DateTime LastAttemptTime { get => _lastAttemptTime; set => _lastAttemptTime = value; } // Добавлен

    public OnlinePayment(int paymentMethodId, string methodName, decimal minAmount, string paymentUrl, string currency, ILogger logger, ITransactionIdGenerator transactionIdGenerator) : base(paymentMethodId, methodName, minAmount, currency, logger, transactionIdGenerator)
    {
        PaymentUrl = paymentUrl;
        AttemptsCount = 0;
    }

    public override void ProcessPayment(decimal amount)
    {
        base.ProcessPayment(amount);
        TransactionId = _transactionIdGenerator.GenerateTransactionId();
        _logger.Log($"Перенаправление на платежную страницу: {PaymentUrl}");
        AttemptsCount++;
        LastAttemptTime = DateTime.Now; // Обновить время последней попытки
    }

    public string GetPaymentDetails(bool showTransactionId = false)
    {
        if (showTransactionId)
        {
            return $"Способ оплаты: {MethodName}, Минимальная сумма: {MinAmount} {Currency}, ID транзакции: {TransactionId}";
        }
        else
        {
            return base.GetPaymentDetails();
        }
    }

    //Добавлен: Способ проверки правильности URL-адреса платежа
    public bool IsPaymentUrlValid()
    {
        //Добавьте сюда свою логику, чтобы проверить, корректен ли URL. Это заполнитель
        return !string.IsNullOrEmpty(PaymentUrl) && PaymentUrl.StartsWith("https://");
    }
}

// Производный класс БанковскийПеревод
public class BankTransfer : PaymentMethod
{
    private string _bankData;
    private decimal _bankFee;
    private string _bankName;
    private bool _isBankDataVerified;
    private string _bankTransferReference; // Добавлена: Ссылка на банковский перевод


    public string BankData { get => _bankData; set => _bankData = value; }
    public decimal BankFee { get => _bankFee; set => _bankFee = value; }
    public string BankName { get => _bankName; set => _bankName = value; }
    public bool IsBankDataVerified { get => _isBankDataVerified; set => _isBankDataVerified = value; }
    public string BankTransferReference { get => _bankTransferReference; set => _bankTransferReference = value; } // Добавлено


    public BankTransfer(int paymentMethodId, string methodName, decimal minAmount, string bankData, decimal bankFee, string bankName, string currency, ILogger logger, ITransactionIdGenerator transactionIdGenerator) : base(paymentMethodId, methodName, minAmount, currency, logger, transactionIdGenerator)
    {
        BankData = bankData;
        BankFee = bankFee;
        BankName = bankName;
        IsBankDataVerified = false;
    }

    public override bool CheckMinimumAmount(decimal amount)
    {
        if (amount < MinAmount + BankFee)
        {
            _logger.Log($"Сумма платежа меньше минимальной ({MinAmount} + {BankFee} {Currency})");
            return false;
        }
        return true;
    }

    public override string GetPaymentDetails()
    {
        return $"Способ оплаты: {MethodName}, Минимальная сумма: {MinAmount} {Currency}, Банковские данные: {BankData}, Банк: {BankName}, Комиссия: {BankFee}";
    }

    public string GetBankInfo()
    {
        return $"Название банка: {BankName}, Банковские данные: {BankData}";
    }

    public void VerifyBankData()
    {
        _logger.Log("Проверка банковских данных...");
        IsBankDataVerified = true;
    }

    //Добавлен: Способ создания ссылки на банковский перевод
    public void GenerateBankTransferReference()
    {
        BankTransferReference = Guid.NewGuid().ToString();
        _logger.Log($"Bank transfer reference generated: {BankTransferReference}");
    }
}

// Производный класс Наличные
public class CashPayment : PaymentMethod
{
    private string _cashPickupPoint;
    private string _paymentReference;
    private bool _isPaymentRecorded;
    private string _employeeID; // Добавлен: Идентификатор сотрудника, который осуществлял оплату наличными


    public string CashPickupPoint { get => _cashPickupPoint; set => _cashPickupPoint = value; }
    public string PaymentReference { get => _paymentReference; set => _paymentReference = value; }
    public bool IsPaymentRecorded { get => _isPaymentRecorded; set => _isPaymentRecorded = value; }
    public string EmployeeID { get => _employeeID; set => _employeeID = value; } // Добавлено

    public CashPayment(int paymentMethodId, string methodName, decimal minAmount, string cashPickupPoint, string currency, ILogger logger, ITransactionIdGenerator transactionIdGenerator)
        : base(paymentMethodId, methodName, minAmount, currency, logger, transactionIdGenerator)
    {
        CashPickupPoint = cashPickupPoint;
        IsPaymentRecorded = false;
    }

    public override string GetPaymentDetails()
    {
        return $"Способ оплаты: {MethodName}, Минимальная сумма: {MinAmount} {Currency}, Место выдачи наличных: {CashPickupPoint}";
    }

    public string GetPaymentDetails(bool showReference = false)
    {
        if (showReference)
        {
            return $"Способ оплаты: {MethodName}, Минимальная сумма: {MinAmount} {Currency}, Место выдачи наличных: {CashPickupPoint}, Номер платежа: {PaymentReference}";
        }
        else
        {
            return base.GetPaymentDetails();
        }
    }

    public void RecordPayment(string paymentReference)
    {
        PaymentReference = paymentReference;
        _logger.Log($"Платеж записан с номером: {paymentReference}");
        IsPaymentRecorded = true;
    }

    // Добавлен: Способ присвоения идентификатора сотрудника при оплате наличными.
    public void AssignEmployee(string employeeID)
    {
        EmployeeID = employeeID;
        _logger.Log($"Employee {employeeID} assigned to cash payment.");
    }
}

// Дополнительный класс - PrepaidCard (сложное наследование)
public class PrepaidCard : OnlinePayment
{
    private string _cardNumber;
    private DateTime _cardExpiryDate;
    private bool _isCardVerified;
    private string _cardHolderName; // Добавлено: Имя владельца карты

    public string CardNumber { get => _cardNumber; set => _cardNumber = value; }
    public DateTime CardExpiryDate { get => _cardExpiryDate; set => _cardExpiryDate = value; }
    public bool IsCardVerified { get => _isCardVerified; set => _isCardVerified = value; }
    public string CardHolderName { get => _cardHolderName; set => _cardHolderName = value; } // Добавлено


    public PrepaidCard(int paymentMethodId, string methodName, decimal minAmount, string paymentUrl, string cardNumber, DateTime cardExpiryDate, string currency, ILogger logger, ITransactionIdGenerator transactionIdGenerator) : base(paymentMethodId, methodName, minAmount, paymentUrl, currency, logger, transactionIdGenerator)
    {
        CardNumber = cardNumber;
        CardExpiryDate = cardExpiryDate;
        IsCardVerified = false;
    }

    public override void ProcessPayment(decimal amount)
    {
        base.ProcessPayment(amount);
        _logger.Log($"Используется карта: {CardNumber}, Срок действия: {CardExpiryDate:dd/MM/yyyy}");
    }

    public bool IsCardExpired()
    {
        return CardExpiryDate < DateTime.Now;
    }

    public void VerifyCard()
    {
        _logger.Log("Проверка карты...");
        IsCardVerified = true;
    }

    //Добавлен: Способ установки имени владельца карты
    public void SetCardHolderName(string cardHolderName)
    {
        CardHolderName = cardHolderName;
        _logger.Log($"Card holder name set to: {cardHolderName}");
    }
}

// Интерфейс для множественного наследования
public interface ICommissionable
{
    decimal Commission { get; set; }
}

// Класс BankTransfer, реализующий интерфейс ICommissionable
public class BankTransferWithCommission : BankTransfer, ICommissionable
{
    public decimal Commission { get; set; }

    public BankTransferWithCommission(int paymentMethodId, string methodName, decimal minAmount, string bankData, decimal bankFee, string bankName, string currency, ILogger logger, ITransactionIdGenerator transactionIdGenerator) : base(paymentMethodId, methodName, minAmount, bankData, bankFee, bankName, currency, logger, transactionIdGenerator)
    {
        
    }
}

// Generic класс для обработки платежей
public class PaymentProcessor<T> where T : IPaymentMethod
{
    public void ProcessPayment(T paymentMethod, decimal amount)
    {
        if (paymentMethod.CheckMinimumAmount(amount))
        {
            paymentMethod.ProcessPayment(amount);

            if (paymentMethod is ICommissionable commissionablePaymentMethod)
            {
                Console.WriteLine($"Комиссия: {commissionablePaymentMethod.Commission}");
            }
        }
        else
        {
            Console.WriteLine("Сумма платежа меньше минимальной.");
        }
    }
}

// Класс для хранения истории платежей
public class PaymentHistory
{
    public int PaymentId { get; set; }
    public string PaymentMethod { get; set; }
    public decimal Amount { get; set; }
    public DateTime PaymentDate { get; set; }
}


ILogger logger = new ConsoleLogger();
ITransactionIdGenerator transactionIdGenerator = new GuidTransactionIdGenerator();

OnlinePayment onlinePayment = new OnlinePayment(1, "Онлайн оплата", 100, "https://example.payment.com", "RUB", logger, transactionIdGenerator);
BankTransfer bankTransfer = new BankTransfer(2, "Банковский перевод", 25, "Счет: 0987654321", 10, "ПриватБанк", "RUB", logger, transactionIdGenerator);
CashPayment cashPayment = new CashPayment(3, "Наличные", 50, "Офис компании", "RUB", logger, transactionIdGenerator);
PrepaidCard prepaidCard = new PrepaidCard(4, "Оплата картой", 30, "https://example.payment.com", "00998877665544332211", DateTime.Now.AddYears(1), "RUB", logger, transactionIdGenerator);
BankTransferWithCommission bankTransferWithCommission = new BankTransferWithCommission(5, "Банковский перевод с комиссией", 25, "Счет: 9876543210", 10, "Сбербанк", "RUB", logger, transactionIdGenerator);
bankTransferWithCommission.Commission = 25;

PaymentProcessor<IPaymentMethod> paymentProcessor = new PaymentProcessor<IPaymentMethod>();

paymentProcessor.ProcessPayment(onlinePayment, 200);
paymentProcessor.ProcessPayment(bankTransfer, 150);
paymentProcessor.ProcessPayment(cashPayment, 75);
paymentProcessor.ProcessPayment(prepaidCard, 200);
paymentProcessor.ProcessPayment(bankTransferWithCommission, 100);
   


Обработка платежа методом Онлайн оплата на сумму 200 RUB
Перенаправление на платежную страницу: https://example.payment.com
Обработка платежа методом Банковский перевод на сумму 150 RUB
Обработка платежа методом Наличные на сумму 75 RUB
Обработка платежа методом Оплата картой на сумму 200 RUB
Перенаправление на платежную страницу: https://example.payment.com
Используется карта: 00998877665544332211, Срок действия: 01/12/2025
Обработка платежа методом Банковский перевод с комиссией на сумму 100 RUB
Комиссия: 25
